# Synchronous vs Asynchronous Code

- Following code is an Asynchronous code, because its output after execution is: 

**Before**

**After**

**Reading user from database**

- Asynchronous does not mean concurrent or multi-threaded. In this app we have a single thread that executes first line, then schedules a function to be called after two seconds and finally executes the third line.

In [ ]:
console.log('Before');
setTimeout(() => {
    console.log('Reading user from database');
}, 2000)
console.log('After');

# Patterns for Dealing with Asynchronous Code

- In the following code, third line (**console.log(user);**) shows undefined, because user constant will be set after two seconds.
- **setTimeout** function simulates a long running operation like database reading. In this kind of operations the result is not available immediatly.
- We have three patterns to deal with asynchronous code: **Callbacks, Promises, Async/await**

In [2]:
console.log('Before');
const user = getUser(1);
console.log(user);
console.log('After');

function getUser(id){
    setTimeout(() => {
        console.log('Reading user from database');
        return { id: id, githubusername: 'mosh' };
    }, 2000)
}

Before
undefined
After


undefined

Reading user from database


# Callbacks

- A **callback** is a function that we are going to call when the result of an asynchronous operation is ready. 
- In this app we use callback function that returns the user object and passes as argument to other function. In this case the other function is an argument of the *getUser* function.

In [3]:
console.log('Before');
getUser(1, (user) => {
    console.log('User: ', user)
});
console.log('After');

function getUser(id, callback){
    setTimeout(() => {
        console.log('Reading user from database');
        callback({ id: id, githubusername: 's-gil' });
    }, 2000)
}

Before
After


undefined

Reading user from database
User:  { id: 1, githubusername: 's-gil' }


In [5]:
// Exercise

console.log('Before');
getUser(1, (user) => {
    console.log('User: ', user)
    
    getRepositories(user.gitHubUsername, (repos) => {
        console.log('Repositories: ', repos);
    });
});
console.log('After');

function getUser(id, callback){
    setTimeout(() => {
        console.log('Reading user from database');
        callback({ id: id, gitHubUsername: 's-gil' });
    }, 2000)
}

function getRepositories(username, callback) {
    setTimeout(() => {
        console.log('Reading Repositories');
        callback(['repo1', 'repo2', 'repo3']);
    }, 2000)    
}


Before
After


undefined

Reading user from database
User:  { id: 1, gitHubUsername: 's-gil' }
Reading Repositories
Repositories:  [ 'repo1', 'repo2', 'repo3' ]


# Callback Hell Problem

In [ ]:
getUser(1, (user) => {    
    getRepositories(user.gitHubUsername, (repos) => {
        getCommits(repos[], () => {
            ... // Nested structure
        });
    });
});

- In past code we built nested structure that is difficult to read. It is called **CALLBACK HELL** or *christmas tree problem* 

# Named Functions to Rescue

- **Anonymous function** is a function that does not have a name. For example, in **getUser** the second argument an anonymous function.
- We are going to replace each anonymous function with a **named function**

In [30]:
// Exercise

console.log('Before');
getUser(1, getRepositories);
console.log('After');

function getRepositories(user){
    getRepositories(user.gitHubUsername, getCommits);
}

function getCommits(repos){
    getCommits(repos, displayCommits);
}

function displayCommits(commits){
    console.log('Commits: ', commits);
}

function getUser(id, callback){
    setTimeout(() => {
        console.log('Reading user from database');
        callback({ id: id, gitHubUsername: 's-gil' });
    }, 2000)
}

function getRepositories(username, callback) {
    setTimeout(() => {
        console.log('Reading Repositories');
        callback(['repo1', 'repo2', 'repo3']);
    }, 2000)    
}

function getCommits(repos, callback) {
    setTimeout(() => {
        console.log('Reading Commits');
        callback(['commit1', 'commit2', 'commit3']);
    }, 2000)    
}


Before
After


undefined

Reading user from database
Reading Repositories


TypeError: callback is not a function
    at Timeout._onTimeout (evalmachine.<anonymous>:29:9)
    at listOnTimeout (node:internal/timers:559:17)
    at processTimers (node:internal/timers:502:7)

- Previous code does  not work (https://stackoverflow.com/questions/66672011/typeerror-callback-is-not-a-function-node-js).

**Solution:**

In [29]:
// Exercise

console.log('Before');
getUser(1, getRepositories);
console.log('After');

function displayCommits(commits){
    console.log('Commits: ', commits);
}

function getUser(id, callback){
    setTimeout(() => {
        console.log('Reading user from database');
        const user = { id: id, gitHubUsername: 's-gil' };
        callback(user.gitHubUsername, getCommits);
//         callback({ id: id, gitHubUsername: 's-gil' });
    }, 2000)
}

function getRepositories(username, callback) {
    setTimeout(() => {
        console.log('Reading Repositories');
        const repos = ['repo1', 'repo2', 'repo3']
        callback(repos[0], displayCommits);
    }, 2000)    
}

function getCommits(repos, callback) {
    setTimeout(() => {
        console.log('Reading Commits');
        const commits = ['commit1', 'commit2', 'commit3']
        callback(commits);
    }, 2000)    
}


Before
After


undefined

Reading user from database
Reading Repositories
Reading Commits
Commits:  [ 'commit1', 'commit2', 'commit3' ]


# Promises

- promise is an object that holds the eventual result of an asynchronous operation. It can be in one of three states

Pending: call some asynchronous operation (object creation).
Fulfilled: asynchronous operation completed succesfully.
Rejected: something went wrong.

- We create a new file named **promise.js**

In [35]:
//promise.js
//---------------------------------------------------------
const p = new Promise((resolve, reject) => {
    //Async Work...
    //.
    //.
    //.
    
    resolve(1); // asynchronous operation completed succesfully
});

p.then(result => console.log('Result', result))

Promise { <pending> }

Result 1


In [38]:
const p = new Promise((resolve, reject) => {
    //Async Work...
    setTimeout(() => {
        resolve(1);
    }, 2000);
});

p.then(result => console.log('Result', result))

Promise { <pending> }

Result 1


In [42]:
const p = new Promise((resolve, reject) => {
    //Async Work...
    setTimeout(() => {
//         resolve(1) //resilved, fulfilled
        reject(new Error('message')); // rejected
    }, 2000);
});

// consuming...
p
    .then(result => console.log('Result', result))
    .catch(err => console.log('Error', err.message));


Promise { <pending> }

Error message


# Replacing callbacks with promises

In [ ]:

// function getUser(id, callback){
//     setTimeout(() => {
//         console.log('Reading user from database');
//         const user = { id: id, gitHubUsername: 's-gil' };
//         callback({ id: id, gitHubUsername: 's-gil' });
//     }, 2000)
// }

function getUser(id){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading user from database');
            resolve({ id: id, gitHubUsername: 's-gil' });
        }, 2000);
    });    
}

function getRepositories(username) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Repositories');
            resolve(['repo1', 'repo2', 'repo3']);
        }, 2000);
    });        
}

function getCommits(repos) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Commits');
            resolve(['commit1', 'commit2', 'commit3']);
        }, 2000);
    });        
}

# Consuming Promises 

In [52]:
//index.js
//----------------------------------------------------------------
console.log('Before');
const p = getUser(1);
p.then(user => console.log(user));
console.log('After');

function getUser(id){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading user from database');
            resolve({ id: id, gitHubUsername: 's-gil' });
        }, 2000);
    });    
}

function getRepositories(username) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Repositories');
            resolve(['repo1', 'repo2', 'repo3']);
        }, 2000);
    });        
}

function getCommits(repos) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Commits');
            resolve(['commit1', 'commit2', 'commit3']);
        }, 2000);
    });        
}

Before
After


undefined

Reading user from database
{ id: 1, gitHubUsername: 's-gil' }


In [54]:
console.log('Before');
getUser(1)
    .then(user => console.log(user));
console.log('After');

function getUser(id){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading user from database');
            resolve({ id: id, gitHubUsername: 's-gil' });
        }, 2000);
    });    
}

Before
After


undefined

Reading user from database
{ id: 1, gitHubUsername: 's-gil' }


- Callback hell shows a nested structure. Using **promises** we have a flat structure.

In [56]:
//index.js
//----------------------------------------------------------------
console.log('Before');
getUser(1)
    .then(user => getRepositories(user.gitHubUsername))
    .then(repos => getCommits(repos[0]))
    .then(commits => displayCommits(commits))
    .catch(err => console.log('Error', err.message))
console.log('After');

function displayCommits(commits){
    console.log('Commits: ', commits);
}

function getUser(id){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading user from database');
            resolve({ id: id, gitHubUsername: 's-gil' });
        }, 2000);
    });    
}

function getRepositories(username) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Repositories');
            resolve(['repo1', 'repo2', 'repo3']);
        }, 2000);
    });        
}

function getCommits(repos) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Commits');
            resolve(['commit1', 'commit2', 'commit3']);
        }, 2000);
    });        
}

Before
After


undefined

Reading user from database
Reading Repositories
Reading Commits
Commits:  [ 'commit1', 'commit2', 'commit3' ]


# Creating Settled Promises

- Create a promise that is already resolved. This is useful when writing unit test.
- We want to simulate an scenario where asynchronous operation completes succesfully.
- we create a file named **promise-api.js**

In [61]:
//promise-api.js
//---------------------------------------------
const p = Promise.resolve({ id: 1 }); // return a promise already resolved
p.then(result => console.log(result));

Promise { <pending> }

{ id: 1 }


In [60]:
//promise-api.js
//---------------------------------------------
const p = Promise.reject(new Error('reason for rejection...')); // return a promise already rejected
p.catch(error => console.log(error));

Promise { <pending> }

Error: reason for rejection...
    at evalmachine.<anonymous>:3:26
    at Script.runInThisContext (node:vm:129:12)
    at Object.runInThisContext (node:vm:305:38)
    at run ([eval]:1020:15)
    at onRunRequest ([eval]:864:18)
    at onMessage ([eval]:828:13)
    at process.emit (node:events:527:28)
    at emit (node:internal/child_process:938:14)
    at processTicksAndRejections (node:internal/process/task_queues:84:21)


# Running Promises in Parallel

- **Promise.all** method get an array of promises and return a new promise that will be resolve when all the promises are resolved.
- Here we don´t have real concurrency or multi-threading. We have a single thread  kicking off multiple asynchronous operations almost at the same time.
- If one the promise fail, the Promise.all is reject


In [64]:
//promise-api.js
//---------------------------------------------

const p1 = new Promise((resolve) => {
    setTimeout(()=>{
        console.log('Async opeartion 1...');
        resolve(1);
    }, 2000);
});

const p2 = new Promise((resolve) => {
    setTimeout(()=>{
        console.log('Async opeartion 2...');
        resolve(2);
    }, 2000);
});

Promise.all([p1,p2])
    .then(result => console.log(result));

Promise { <pending> }

Async opeartion 1...
Async opeartion 2...
[ 1, 2 ]


- With **promise.race**, as soon as one promise is fulfilled the promise returned by race method will be considered fulfilled. The result is the value of the first fulfilled promise.

In [65]:
//promise-api.js
//---------------------------------------------
const p1 = new Promise((resolve) => {
    setTimeout(()=>{
        console.log('Async opeartion 1...');
        resolve(1);
    }, 2000);
});

const p2 = new Promise((resolve) => {
    setTimeout(()=>{
        console.log('Async opeartion 2...');
        resolve(2);
    }, 2000);
});

Promise.race([p1,p2])
    .then(result => console.log(result));

Promise { <pending> }

Async opeartion 1...
1
Async opeartion 2...


# Async and Await

- Async and Await helps us write asynchronous code like synchronous code.

In [ ]:
//Promised-based approach
getUser(1)
    .then(user => getRepositories(user.gitHubUsername))
    .then(repos => getCommits(repos[0]))
    .then(commits => displayCommits(commits))
    .catch(err => console.log('Error', err.message))

//Async and Await approach
const user = await getUser(1);
const repos = await getRepositories(user.gitHubUserName);
const commits = await getCommits(repos[0]);
console.log(commits);

- When using the **await** operator in a function, we need to decorate that function with the **async** modifier. 
- the returned type of this function is a promise that once fulfilled does not resolve in a value, it is void. This means that async and await are built in top of promises 

In [ ]:
//Promised-based approach
getUser(1)
    .then(user => getRepositories(user.gitHubUsername))
    .then(repos => getCommits(repos[0]))
    .then(commits => displayCommits(commits))
    .catch(err => console.log('Error', err.message))

//Async and Await approach
async function displayCommits(){
    const user = await getUser(1);
    const repos = await getRepositories(user.gitHubUserName);
    const commits = await getCommits(repos[0]);
    console.log(commits);
}

In [68]:
//index.js
//----------------------------------------------------------------
console.log('Before');

// //Promised-based approach
// getUser(1)
//     .then(user => getRepositories(user.gitHubUsername))
//     .then(repos => getCommits(repos[0]))
//     .then(commits => displayCommits(commits))
//     .catch(err => console.log('Error', err.message))

//Async and Await approach
async function displayCommits(){
    const user = await getUser(1);
    const repos = await getRepositories(user.gitHubUserName);
    const commits = await getCommits(repos[0]);
    console.log(commits);
}

displayCommits()
console.log('After');

function getUser(id){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading user from database');
            resolve({ id: id, gitHubUsername: 's-gil' });
        }, 2000);
    });    
}

function getRepositories(username) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Repositories');
            resolve(['repo1', 'repo2', 'repo3']);
        }, 2000);
    });        
}

function getCommits(repos) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Commits');
            resolve(['commit1', 'commit2', 'commit3']);
        }, 2000);
    });        
}

Before
After


undefined

Reading user from database
Reading Repositories
Reading Commits
[ 'commit1', 'commit2', 'commit3' ]


- When using async and await we don´t have catch method. To get error we use **try-catch** block

In [69]:
//index.js
//----------------------------------------------------------------
console.log('Before');

//Async and Await approach
async function displayCommits(){
    try{
        const user = await getUser(1);
        const repos = await getRepositories(user.gitHubUserName);
        const commits = await getCommits(repos[0]);
        console.log(commits);
    }
    catch(err){
        console.log('Error', err.message);
    }
}

displayCommits()
console.log('After');

function getUser(id){
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading user from database');
            resolve({ id: id, gitHubUsername: 's-gil' });
        }, 2000);
    });    
}

function getRepositories(username) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Repositories');
//             resolve(['repo1', 'repo2', 'repo3']);
            reject(new Error('Could not get the repos')) //simulating error
        }, 2000);
    });        
}

function getCommits(repos) {
    return new Promise((resolve, reject) => {
        setTimeout(() => {
            console.log('Reading Commits');
            resolve(['commit1', 'commit2', 'commit3']);
        }, 2000);
    });        
}

Before
After


undefined

Reading user from database
Reading Repositories
Error Could not get the repos


# Exercise

In [74]:
// Replace with await and async approach

getCustomer(1, (customer) => {
  console.log('Customer: ', customer);
  if (customer.isGold) {
    getTopMovies((movies) => {
      console.log('Top movies: ', movies);
      sendEmail(customer.email, movies, () => {
        console.log('Email sent...')
      });
    });
  }
});

function getCustomer(id, callback) {
  setTimeout(() => {
    callback({ 
      id: 1, 
      name: 'Sebastian Gil', 
      isGold: true, 
      email: 'email' 
    });
  }, 4000);  
}

function getTopMovies(callback) {
  setTimeout(() => {
    callback(['movie1', 'movie2']);
  }, 4000);
}

function sendEmail(email, movies, callback) {
  setTimeout(() => {
    callback();
  }, 4000);
}

undefined

Customer:  { id: 1, name: 'Sebastian Gil', isGold: true, email: 'email' }
Top movies:  [ 'movie1', 'movie2' ]
Email sent...


In [77]:
// Solution

// getCustomer(1, (customer) => {
//   console.log('Customer: ', customer);
//   if (customer.isGold) {
//     getTopMovies((movies) => {
//       console.log('Top movies: ', movies);
//       sendEmail(customer.email, movies, () => {
//         console.log('Email sent...')
//       });
//     });
//   }
// });

async function func(){
    const customer = await getCustomer(1);
    console.log(customer);
    if (customer.isGold){
        const topMovies = await getTopMovies();
        console.log(topMovies);
        await sendEmail(customer.email, topMovies); 
        console.log('Email sent...');
    }    
    
}

func();

function getCustomer(id) {
    return new Promise((resolve) => {
        setTimeout(() => {
            resolve({ 
            id: 1, 
            name: 'Sebastian Gil', 
            isGold: true, 
            email: 'email'
            });
        }, 4000);
    });  
}

function getTopMovies() {
    return new Promise((resolve) => {
       setTimeout(() => {
           resolve(['movie1', 'movie2']);
       }, 4000); 
    });   
}

function sendEmail(email, movies) {
    return new Promise((resolve) => {
        setTimeout(() => {
            resolve();
        }, 4000);
    });
}


Promise { <pending> }

{ id: 1, name: 'Sebastian Gil', isGold: true, email: 'email' }
[ 'movie1', 'movie2' ]
Email sent...
